# `PointBallot`
While some elections rely on candidate rankings, others rely on candidate scorings, such as Borda, Cumulative, or approval voting. The `PointBallot` class stores such scoring information as a dictionary, where each key is a candidate and each value is the number of points given to that candidate by the voter.

The `points` attribute can be either a list with multiplicity, or a dictionary.

In [1]:
import votekit.ballot_generator as bg
from votekit.ballot import PointBallot

from votekit.elections import HighestScore
from votekit.pref_profile import PreferenceProfile
from fractions import Fraction

In [2]:
print(PointBallot(points=["A", "B", "A"]))
print(PointBallot(points={"A":2, "B":1}, weight = Fraction(3,1)))

{'A': 2, 'B': 1} with weight 1
{'A': 2, 'B': 1} with weight 3


# Cumulative Voting

In a cumulative election, voters have $m$ votes to fill $m$ seats. They are allowed to cast these votes on any candidate, including multiple votes for one candidate. Then, the candidates with the $m$ highest vote totals win. We still model this using a preference interval, except now we just sample with replacement (instead of without replacement as in the PL model).

In [3]:
candidates = ["A", "B", "C"]
pref_interval_by_bloc = {"R": {"A": .3, "B":.6, "C":.1}}
bloc_voter_prop = {"R":1}
num_seats = 2

cumulative = bg.Cumulative( num_votes=num_seats, candidates = candidates, pref_interval_by_bloc = pref_interval_by_bloc,
                           bloc_voter_prop=bloc_voter_prop)



In [4]:
cumulative_profile = cumulative.generate_profile(number_of_ballots=10)
cumulative_election = HighestScore(profile = cumulative_profile,
                                 seats = num_seats)

print(cumulative_profile)

     Ballots  Weight
     (B: 2,)       4
(A: 1, B: 1)       3
     (A: 2,)       2
(B: 1, C: 1)       1


In [5]:
cumulative_election.run_election()

Candidate     Status  Round
        B    Elected      1
        A    Elected      1
        C Eliminated      1

Double check the computation above!


Now what happens in the case of ties? As currently coded, `votekit` takes all the tied candidates and randomly permutes them.

In [6]:
tied_cumulative_profile = PreferenceProfile(ballots = [PointBallot(points=["A", "B", "C"])],
                                       candidates = ["A", "B", "C"])

In [8]:
cumulative_election = HighestScore(profile = tied_cumulative_profile,
                                 seats = num_seats)
cumulative_election.run_election()

A tie was broken.


Candidate     Status  Round
        B    Elected      1
        C    Elected      1
        A Eliminated      1